In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-
"""This notebook builds the CSV-like Wikimedia page which records the contributions to WLE in Spain in YEAR"""

import os, sys, inspect

try :
    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[0:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

from io import StringIO
import pandas as pd
import numpy as np
import re

from datetime import datetime, timedelta

In [ ]:
YEAR                = 2017
TAG                 = 'WLE'
TAG_EXT             = 'Wiki Loves Earth'
TEMPLATE            = 'lic'

BASE_NAME           = "Commons:Wiki Loves in Spain/{1}/{0}".format(YEAR, TAG_EXT)
LOG_PAGE            = BASE_NAME + u"/Log"

WLE_CATEGORY          = "Category:Images from {1} {0} in Spain".format(YEAR, TAG_EXT)
WLE_FINALIST_CATEGORY = "Category:Evaluation of images from {1} {0} in Spain - Final".format(YEAR, TAG_EXT)

commons_site = pb.Site("commons", "commons")

### Image selection

In [ ]:
cat_wle = pb.Category(commons_site, WLE_CATEGORY)
gen_wle = pagegenerators.CategorizedPageGenerator(cat_wle)

images_wle = [page.title(withNamespace=False) for page in gen_wle if page.is_filepage()]
len(images_wle)

### Image review and recording

In [ ]:
images_df = pd.DataFrame(
    columns=['image_title', 'lic_id', 'uploader', 'uploader_registration', 
             'timestamp', 'date', 'size', 'height', 'width', 'qi', 'finalist'])

uploaders = {}

for image_counter, image in enumerate(images_wle):
    if (image_counter != 0) and (image_counter % 50 == 0) :
        pb.output ('Retrieving --> %d image descriptions downloaded' %(image_counter))
    image_row = {'image_title': None, 
                 'lic_id': '', 
                 'uploader': None,
                 'uploader_registration': None,
                 'timestamp': None,
                 'date': None,
                 'size': 0,
                 'height': 0,
                 'width': 0,
                 'qi': None,
                 'finalist': None}

    page = pb.FilePage(commons_site, image)
    image_row["image_title"] = page.title(withNamespace=False)
    
    fileinfo = page.latest_file_info
    image_row["size"] = fileinfo['size']
    image_row["height"] = fileinfo['height']
    image_row["width"] = fileinfo['width']
    
    text = page.text
    wikicode = mwh.parse(text)
    templates = wikicode.filter_templates()

    info_templates = [template for template in wikicode.filter_templates() 
                      if template.name.lower().strip() == "information"]
    
    # code management
    wle_templates = [template for template in wikicode.filter_templates() 
                    if template.name.lower().strip() == TEMPLATE]
    if len(wle_templates) > 0:
        image_row["lic_id"] = wle_templates[0].get(1).value.strip()
    
    # date management
    if info_templates[0].has("date"):
        try :
            datetime_object = datetime.strptime(info_templates[0].get("date").value.strip().split(' ')[0], '%Y-%m-%d')
            image_row["date"] = info_templates[0].get("date").value.strip().split(' ')[0]
        except :
            print ("Image with wrong date ({0})".format(image_row["image_title"]))
            pass
    elif info_templates[0].has("Date"):
        try :
            datetime_object = datetime.strptime(info_templates[0].get("Date").value.strip().split(' ')[0], '%Y-%m-%d')
            image_row["date"] = info_templates[0].get("Date").value.strip().split(' ')[0]
        except :
            print ("Image with wrong date ({0})".format(image_row["image_title"]))
            pass
    else :
        print ("Image with no date ({0})".format(image_row["image_title"]))

    # uploader management
    creation = page.oldest_revision
    image_row["uploader"] = creation["user"]
    
    # uploader registration management
    if image_row["uploader"] not in uploaders :
        try:
            # Too old users do not have a user registration time
            user_registration = pb.User(commons_site, title=creation["user"]).registration()
            image_row["uploader_registration"] = user_registration.strftime("%Y-%m-%d")
        except :
            image_row["uploader_registration"] = '2006-01-01'
        uploaders[image_row["uploader"]] = image_row["uploader_registration"]
    else :
        image_row["uploader_registration"] = uploaders[image_row["uploader"]]
        
    # upload time management
    creation_time = creation.timestamp + timedelta(hours=2)
    image_row["timestamp"] = creation_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # quality status management
    qi_templates = [template for template in wikicode.filter_templates() 
                   if template.name.lower().strip() == "qualityimage"]    
    if len(qi_templates) > 0:
        image_row["qi"] = 'qi'
        
    # finalist status management
    cats = [cat.title(withNamespace=True) for cat in textlib.getCategoryLinks(text)]
    if WLE_FINALIST_CATEGORY in cats:
        image_row["finalist"] = 'finalist'

    images_df = images_df.append(image_row, ignore_index=True)

In [ ]:
images_df.describe(include="all")

### Log creation

In [ ]:
buf = StringIO()
images_df.to_csv(buf, index=None, sep=';', encoding='utf-8', header=False)

db_page = pb.Page(commons_site, LOG_PAGE)
db_page.text = u'<pre>\n' + buf.getvalue() + u'</pre>'
db_page.save("{1} {0} in Spain: Contribution log update".format(YEAR, TAG))